<a href="https://colab.research.google.com/github/kampelmuehler/MLKurs/blob/main/image_recognition/SemanticSegmentationDemo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
#@title Setup
import torch
from torchvision import models
from PIL import Image
import urllib
import ast
import matplotlib.pyplot as plt
import numpy as np


weights = models.segmentation.DeepLabV3_ResNet101_Weights.DEFAULT
transform = models.segmentation.DeepLabV3_ResNet101_Weights.COCO_WITH_VOC_LABELS_V1.transforms()
model = models.segmentation.deeplabv3_resnet101(weights=weights)
model.eval()
labels = ast.literal_eval(urllib.request.urlopen("https://raw.githubusercontent.com/kampelmuehler/MLKurs/main/image_recognition/VOC_classes.txt").read().decode('utf-8'))


def read_image(url):
  req = urllib.request.Request(
    url, 
    data=None, 
    headers={
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'
    }
  )
  img = Image.open(urllib.request.urlopen(req)).convert("RGB")
  thumbnail = img.copy()
  thumbnail.thumbnail((400, 400), Image.ANTIALIAS)
  display(thumbnail)
  return transform(img).unsqueeze(0), img, thumbnail.size

palette = torch.tensor([2 ** 25 - 1, 2 ** 15 - 1, 2 ** 21 - 1])
colors = torch.as_tensor([i for i in range(21)])[:, None] * palette
colors = (colors % 255).numpy().astype("uint8")

# COCO VOC Subset Semantische Segmentierung

Hier könnt ihr ein auf MobileNetv3 basierendes DeepLabV3 Modell, welches auf einem COCO subset mit VOC labels trainiert ist, ausprobieren. Einfach eine url zu einem Bild einfügen.

Die Klassen finden sie in der Variable `labels`.

In [ ]:
url = ""
img_tensor, img, thumbnail_size = read_image(url)
result = model(img_tensor)['out'][0].argmax(0)

result_array = result.byte().cpu().numpy()
output = "Gefundene Klassen: "
for c in np.unique(result_array):
  output += f"{labels[c]}, "
print(output[:-2])
result_img = Image.fromarray(result_array).resize(thumbnail_size)
result_img.putpalette(colors)
result_img = result_img.convert("RGBA")
img = img.resize(thumbnail_size).convert("RGBA")
display(Image.blend(img, result_img, 0.7))